In [8]:
from flask import Flask, request, Blueprint
import pymongo
import numpy as np
import json
from bson import ObjectId
import base64
from pymongo.database import Database 
from datetime import date, datetime
from datetime import datetime
from pyisemail import is_email
import smtplib
from email.mime.text import MIMEText
from email.utils import formataddr
from random import randint
from operator import itemgetter
from collections import Counter
import matplotlib.pyplot as plt
from io import BytesIO

In [9]:
def mongodb_init():
    # connect to mongodb
    mongo = pymongo.MongoClient(host='18.162.214.19', port=27017, username="root", password="1647#4hkust",
                                authSource='admin')
    print('数据库当前的databases: ', mongo.list_database_names())
    return mongo


def get_db(mongo, db_name):
    db = Database(name=db_name, client=mongo)
    print('获取/创建库：', db.name)
    return db


def get_time_attribute(type):
    # generate current time
    now_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    if type == 'update':
        return {'UpdateTime': now_time}
    elif type == 'create':
        return {'CreateTime': now_time, 'UpdateTime': now_time, 'CodeTime': now_time}
    elif type == 'create withour code':
        return {'CreateTime': now_time, 'UpdateTime': now_time}
    elif type == 'only create':
        return {'CreateTime': now_time}
    elif type == 'code':
        return {'UpdateTime': now_time, 'CodeTime': now_time}

    print("Please input type = 'update', 'create', 'create withour code', 'only create', 'code' ")
    return 0


# API below
app = Flask(__name__)
mongo = mongodb_init()
db = get_db(mongo, 'chen_db')

数据库当前的databases:  ['admin', 'chen_db', 'config', 'local']
获取/创建库： chen_db


In [10]:
def Image_Encode(image_URL):
    # 读取本地图片文件
    with open(image_URL, 'rb') as input:
        image_data = input.read()  # 图片的二进制数据

    # 将二进制图片编码为 base64 格式
    base64_data_bytes = base64.b64encode(image_data)  # 图片的 bytes 类型
    base64_data_str = base64_data_bytes.decode()  # 图片的字符串类型\
    return base64_data_str

In [11]:
def Image_show(image_URL):
    with open(image_URL, 'rb') as input:
        image_data = input.read()  # 图片的二进制数据
    # 展示图片
    img_data = plt.imread(BytesIO(image_data), format="JPG")
    import cv2
    img_cv2 = cv2.cvtColor(img_data, cv2.COLOR_RGB2BGR)
    cv2.imshow('Image', img_cv2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [12]:
imageUrl_1 = '../image/' + '1.webp'
imageUrl_2 = '../image/' + '2.webp'
imageUrl_3 = '../image/' + '3.webp'
imageUrl_4 = '../image/' + '4.webp'
Images = []
Images.append( Image_Encode(imageUrl_1) )
Images.append( Image_Encode(imageUrl_2) )
Images.append( Image_Encode(imageUrl_3) )
Images.append( Image_Encode(imageUrl_4) )

Title = "Samsung Galaxy Z Fold 4 - Samsung Care - With Stylus - Unlocked"

Text = \
"COMES WITH ORIGINAL SAMSUNG CASE WITH STYLUS\n\
Very good Condition bar some usual wear and tear marks on the screen. The rest of the phone is very clean and in excellent condition.\n\
It comes with Samsung Care for another year; I'm not sure if it's transferable to the new owner. You'll need to verify yourself. however, it's a genuine sale for a genuine phone.\n\
It will be reset to factory settings, and you can insert your SIM—this costs over $2,000 when new with Samsung Care.\n\
Grab a Bargain\n\
Cash on Collection only\n\
Price is Firm - No Low Ballers as they will be ignored."

Price = "995"

In [17]:
import requests
url = 'http://localhost:8080/new-post'
body = {
            'PostOwner': '1030920919@qq.com',
            'Title': Title,
            'Text': Text,
            'Auction': False,
            'LostFound': False,
            'Price': Price,
            'Fields': [],
            'Images': Images
           }
r = requests.post(url, json=body)
print(r)

<Response [200]>
